In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
from sklearn.metrics import mean_squared_error


In [ ]:
# %run PrepareData.ipynb

In [ ]:
tsla_stock_values_df.info()

In [ ]:
tsla_feature_df = tsla_stock_values_df["close_value"]
tsla_feature_df.info()

In [ ]:
tsla_feature_df.plot()

In [ ]:
minmax_scaler = MinMaxScaler(feature_range = (0,1))
tsla_feature_df = minmax_scaler.fit_transform(np.array(tsla_feature_df).reshape(-1,1))
tsla_feature_df

In [ ]:
# This function is used to pre-process the data, where in we are converting data into dependent and independent data based on timestamp
def create_model_dataset(dataset,time_step):
    XData, YData = [],[]    
    for i in range(len(dataset)-time_step-1):
        t = i + time_step
        data = dataset[i : i+time_step,0]
        XData.append(data)        
        YData.append(dataset[i+time_step,0])
    return np.array(XData),np.array(YData)   


In [ ]:
train_data_size = int (len(tsla_feature_df) * .65)
test_data_size = len(tsla_feature_df) - train_data_size

# print(train_data_size)
# print(test_data_size)
# print(train_data_size + test_data_size)
train_data, test_data = tsla_feature_df[0:train_data_size,:],tsla_feature_df[train_data_size:len(tsla_feature_df),:1 ]

time_step = 200
X_train, y_train = create_model_dataset(train_data,time_step)
X_test, y_test = create_model_dataset(test_data,time_step)

X_train

In [ ]:
# for LSTM we need to reshape our X_train and X_test to be [samples, time steps and batch_size]
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
# Create LSTM model
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(200,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer= 'adam')
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=100,batch_size=64,verbose=1)

In [ ]:
# Prediction
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


In [ ]:
#Transform predicted data to original form
train_predict = minmax_scaler.inverse_transform(train_predict)
test_predict= minmax_scaler.inverse_transform(test_predict)

In [ ]:
# Calculate RMSE for both train and test data
import math
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
# Plot for comparing actual data with train predictions, test predictons
# as we considers time step of 100, going back 100 days to better visualistion

back_time = 200
# shift train predictions

trainpredict_plt = np.empty_like(tsla_feature_df)
trainpredict_plt[:,:] = np.nan
trainpredict_plt[back_time: len(train_predict) + back_time ,: ] = train_predict

# shift test predictions
testpredict_plt = np.empty_like(tsla_feature_df)
testpredict_plt[:,:] = np.nan
testpredict_plt[len(train_predict)+(back_time*2)+1: len(tsla_feature_df)-1,: ] = test_predict

plt.plot(minmax_scaler.inverse_transform(tsla_feature_df))
plt.plot(trainpredict_plt)
plt.plot(testpredict_plt)
plt.legend()
plt.show()